In [1]:
import numpy as np
import pandas as pd
from dask import dataframe as dd
import string
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = nlp.tokenizer
stemmer = SnowballStemmer("english")

pd.set_option('display.max_columns', 100)

# Government Spend Analysis

## Step 1: Examine column headers to identify columns of interest

In [2]:
test = pd.read_csv('FY2016_All_Contracts_Full_20210608_1.csv', nrows = 5)

test

,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,total_dollars_obligated,base_and_exercised_options_value,current_total_value_of_award,base_and_all_options_value,potential_total_value_of_award,disaster_emergency_fund_codes_for_overall_award,outlayed_amount_funded_by_COVID-19_supplementals_for_overall_aw,obligated_amount_funded_by_COVID-19_supplementals_for_overall_a,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,ordering_period_end_date,solicitation_date,awarding_agency_code,awarding_agency_name,awarding_sub_agency_code,awarding_sub_agency_name,awarding_office_code,awarding_office_name,funding_agency_code,funding_agency_name,funding_sub_agency_code,funding_sub_agency_name,funding_office_code,funding_office_name,treasury_accounts_funding_this_award,federal_accounts_funding_this_award,object_classes_funding_this_award,program_activities_funding_this_award,foreign_funding,foreign_funding_description,sam_exception,sam_exception_description,recipient_duns,recipient_name,recipient_doing_business_as_name,cage_code,recipient_parent_duns,...,manufacturer_of_goods,veterinary_hospital,hispanic_servicing_institution,receives_contracts,receives_financial_assistance,receives_contracts_and_financial_assistance,airport_authority,council_of_governments,housing_authorities_public_tribal,interstate_entity,planning_commission,port_authority,transit_authority,subchapter_scorporation,limited_liability_corporation,foreign_owned,for_profit_organization,nonprofit_organization,other_not_for_profit_organization,the_ability_one_program,private_university_or_college,state_controlled_institution_of_higher_learning,1862_land_grant_college,1890_land_grant_college,1994_land_grant_college,minority_institution,historically_black_college,tribal_college,alaskan_native_servicing_institution,native_hawaiian_servicing_institution,school_of_forestry,veterinary_college,dot_certified_disadvantage,self_certified_small_disadvantaged_business,small_disadvantaged_business,c8a_program_participant,historically_underutilized_business_zone_hubzone_firm,sba_certified_8a_joint_venture,highly_compensated_officer_1_name,highly_compensated_officer_1_amount,highly_compensated_officer_2_name,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,last_modified_date
0,8600_8600_0001_0_DU205NC16D02_0,CONT_AWD_0001_8600_DU205NC16D02_8600,1,0,0,8600,"HOUSING AND URBAN DEVELOPMENT, DEPARTMENT OF",DU205NC16D02,0,799750.00,546621.23,799750.00,546621.23,799750.00,546621.23,NaN,NaN,NaN,2016-09-30,2016,2016-09-30 00:00:00,2021-09-29 00:00:00,2021-09-29 00:00:00,NaN,NaN,86,DEPARTMENT OF HOUSING AND URBAN DEVELOPMENT (HUD),8600,"HOUSING AND URBAN DEVELOPMENT, DEPARTMENT OF",86545A,DEPT HUD-CHIEF PROCUREMENT OFFICER,86,DEPARTMENT OF HOUSING AND URBAN DEVELOPMENT (HUD),8600,"HOUSING AND URBAN DEVELOPMENT, DEPARTMENT OF",8630AX,DEPT OF HOUSING URBAN DEVELOPMENT,086-2016/2017-0183-000,086-0183,25.3: Other goods and services from Federal so...,0709: ADMINISTRATIVE EXPENSES,X,NOT APPLICABLE,NaN,NaN,831854596,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC",NaN,NaN,831854596,...,f,f,f,f,f,t,f,f,f,f,f,f,f,f,t,f,t,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,t,f,t,f,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_000...,2019-02-12 12:16:44
1,6800_6800_0001_5_EPR91502_0,CONT_AWD_0001_6800_EPR91502_6800,1,5,0,6800,ENVIRONMENTAL PROTECTION AGENCY,EPR91502,0,0.00,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN,2016-09-30,2016,2016-09-30 00:00:00,2017-09-30 00:00:00,2017-09-30 00:00:00,NaN,NaN,68,ENVIRO

### Columns of interest:

* contract_transaction_unique_key
* contract_award_unique_key
* awarding_agency_code
* awarding_sub_agency_code
* awarding_office_code
* funding_agency_code
* funding_sub_agency_code
* funding_office_code
* recipient_duns
* recipient_name
* recipient_doing_business_as_name
* recipient_parent_duns


## Step 2: Load data

Using dask with only columns of interest included.

In [3]:
columns = ['contract_transaction_unique_key', 'contract_award_unique_key', 'awarding_agency_code', 'awarding_sub_agency_code',\
           'awarding_office_code', 'funding_agency_code', 'funding_sub_agency_code', 'funding_office_code', 'recipient_duns', \
           'recipient_name', 'recipient_doing_business_as_name', 'recipient_parent_duns']

data = dd.read_csv('*.csv', usecols=columns, dtype={'awarding_sub_agency_code': 'object', 'funding_sub_agency_code': 'object', \
                                                    'recipient_doing_business_as_name': 'object', 'recipient_parent_duns': 'object',\
                                                    'funding_agency_code': 'object', 'awarding_office_code': 'object', \
                                                    'funding_office_code': 'object', 'recipient_duns': 'object'})

data.head()

,contract_transaction_unique_key,contract_award_unique_key,awarding_agency_code,awarding_sub_agency_code,awarding_office_code,funding_agency_code,funding_sub_agency_code,funding_office_code,recipient_duns,recipient_name,recipient_doing_business_as_name,recipient_parent_duns
0,8600_8600_0001_0_DU205NC16D02_0,CONT_AWD_0001_8600_DU205NC16D02_8600,86,8600,86545A,086,8600,8630AX,831854596,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC",NaN,831854596
1,6800_6800_0001_5_EPR91502_0,CONT_AWD_0001_6800_EPR91502_6800,68,6800,68HE09,068,6800,68Y000,112621987,INNOVATE INC,NaN,112621987
2,6800_6800_0001_20_EPR51502_0,CONT_AWD_0001_6800_EPR51502_6800,68,6800,68HE05,068,6800,68U000,079493355,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...,NaN,079493355
3,6800_6800_0001_0_EPS21602_0,CONT_AWD_0001_6800_EPS21602_6800,68,6800,68HE02,068,6800,68R000,080265258,CAT-ENSOFTEK,NaN,080265258
4,6800_6800_0001_31_EPS51301_0,CONT_AWD_0001_6800_EPS51301_6800,68,6800,68HE05,068,6800,68U000,178599221,"TETRA TECH EM, INC.",NaN,045224250


### Step 2a: Save as parquet format

(So it's easier to reload if needed)

In [4]:
data.to_parquet('government_spend_dataset.parquet', engine='pyarrow')

(None,)

## Step 3: Get number of initial records

In [5]:
a = data.index.size.compute()

print("There are ", a, " initial records in this dataset.")

There are  4820290  initial records in this dataset.


## Step 4: Create list of unique suppliers (exact)

(and get number of unique suppliers)

In [6]:
suppliers_unique = data.recipient_name.unique().compute().reset_index()

print("There are ", len(suppliers_unique), " unique suppliers (exact match only).")

There are  153462  unique suppliers (exact match only).


In [7]:
suppliers_unique.head()

,index,recipient_name
0,0,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC"
1,1,INNOVATE INC
2,2,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...
3,3,CAT-ENSOFTEK
4,4,"TETRA TECH EM, INC."


### Step 4a: Does recipient_doing_business_as_name have any non-null values?

Answer: yes, hold off on this until next step

In [8]:
data.recipient_doing_business_as_name.dropna().compute()

23                MIT LINCOLN LABORATORY
29                                 U T C
39        SOFTWARE ENGINEERING INSTITUTE
52                    PACIFIC SCIENTIFIC
86                    PACIFIC SCIENTIFIC
                      ...               
18863    TOUCHSTONE NEURORECOVERY CENTER
18864                     PROVIDE A RIDE
18875             PETNET PHARMACEUTICALS
18882        MUSC DEPARTMENT OF MEDICINE
18895                 BIMBO BAKERIES USA
Name: recipient_doing_business_as_name, Length: 94562, dtype: object

## Step 5: Normalize text

Using:

 - list of punctuation from string package
 - custom regex to remove spaces between single character words
 - spacy English tokenizer
 - Snowball stemmer from NLTK
 - English stopwords from NLTK

Returning normalized tokens as list in additional column

In [9]:
stop = ['the', 'and', 'inc', 'llc', 'co', 'company', 'corporation', 'group', 'lda', 'services']
punct = string.punctuation

def remove_punct(text):
    text_str = str(text)
    for c in text_str:
        if c in punct:
            text_str = text_str.replace(c, '')
    return text_str

def remove_whitespace(text_str):
    text_str = re.sub(r'(?<=\s[A-Z0-9])\s*[ ]\s*(?=[A-Z0-9]\s)', '', text_str)
    text_str = re.sub(r'(?<=^[A-Z0-9])\s*[ ]\s*(?=[A-Z0-9])', '', text_str)
    text_str = re.sub(r'(?<=[A-Z0-9])\s*[ ]\s*(?=[A-Z0-9]$)', '', text_str)
    return text_str

def normalize_text(text_str):
    wordlist = []
    for w in tokenizer(text_str):
        if str(w).lower() not in stop and not w.is_punct and not w.is_space:
            wordlist.append(stemmer.stem(str(w).lower()))
    return wordlist

suppliers_unique['recipient_name_norm'] = suppliers_unique.recipient_name.\
apply(remove_punct).apply(remove_whitespace).apply(normalize_text)

suppliers_unique

,index,recipient_name,recipient_name_norm
0,0,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC","[mortgag, assist, acquisit]"
1,1,INNOVATE INC,[innov]
2,2,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...,"[ryba, marin, construct, coterra, contract, a,..."
3,3,CAT-ENSOFTEK,[catensoftek]
4,4,"TETRA TECH EM, INC.","[tetra, tech, em]"
...,...,...,...
153457,153457,"AZORMED - GESTAO AMBIENTAL ACOREANA, LDA","[azorm, gestao, ambient, acoreana]"
153458,153458,JEAN ROB & CO,"[jean, rob]"
153459,153459,"WOODS SERVICES, INC.",[wood]
153460,153460,FLOWERS & ASSOCIATES,"[flower, associ]"


In [10]:
suppliers_unique['recipient_name_str'] = suppliers_unique.recipient_name_norm.agg(lambda x: ','.join(map(str, x)))

suppliers_unique.recipient_name_str.value_counts()[suppliers_unique.recipient_name_str.value_counts() > 1].\
sort_values(ascending = False).to_frame(name = 'Additional matches')

,Additional matches
"advanc,prosthet,orthot",9
analyt,8
ur,6
apprais,6
"advanc,technolog",6
...,...
sweat,2
"air,condit,mainten",2
"dtech,lab",2
"qwaltec,incorpor",2


There were 6,488 additional matches discovered through this simple text normalization.

In [11]:
suppliers_unique[suppliers_unique.recipient_name_str == "analyt"]

,index,recipient_name,recipient_name_norm,recipient_name_str
15135,15135,ANALYTIC SERVICES INC.,[analyt],analyt
19186,19186,"ANALYTICAL SERVICES, INC.",[analyt],analyt
25627,25627,ANALYTIC SERVICES INC,[analyt],analyt
52629,52629,"ANALYTIC SERVICES, INC",[analyt],analyt
67509,67509,"ANALYTICS, INC.",[analyt],analyt
83935,83935,ANALYTICAL SERVICES COMPANY,[analyt],analyt
97088,97088,"ANALYTICAL SERVICES, INC",[analyt],analyt
100001,100001,ANALYTICS CORPORATION,[analyt],analyt


In [12]:
suppliers_unique.recipient_name_str.unique().shape

suppliers_unique_list = suppliers_unique.recipient_name_str.unique()

In [13]:
len(suppliers_unique_list)

146202

With normalized text, **146,202** unique suppliers are found. These new normalized names can be joined to the original df. The right index is a unique id per supplier.

In [14]:
data_norm = data.merge(suppliers_unique, on = 'recipient_name').compute()

data_norm

,contract_transaction_unique_key,contract_award_unique_key,awarding_agency_code,awarding_sub_agency_code,awarding_office_code,funding_agency_code,funding_sub_agency_code,funding_office_code,recipient_duns,recipient_name,recipient_doing_business_as_name,recipient_parent_duns,index,recipient_name_norm,recipient_name_str
0,8600_8600_0001_0_DU205NC16D02_0,CONT_AWD_0001_8600_DU205NC16D02_8600,86,8600,86545A,086,8600,8630AX,831854596,"MORTGAGE ASSISTANCE AND ACQUISITIONS GROUP, LLC",NaN,831854596,0,"[mortgag, assist, acquisit]","mortgag,assist,acquisit"
1,6800_6800_0001_5_EPR91502_0,CONT_AWD_0001_6800_EPR91502_6800,68,6800,68HE09,068,6800,68Y000,112621987,INNOVATE INC,NaN,112621987,1,[innov],innov
2,6800_6800_0002_3_EPR91502_0,CONT_AWD_0002_6800_EPR91502_6800,68,6800,68HE09,068,6800,68Y000,112621987,INNOVATE INC,NaN,112621987,1,[innov],innov
3,6800_6800_0001_20_EPR51502_0,CONT_AWD_0001_6800_EPR51502_6800,68,6800,68HE05,068,6800,68U000,079493355,RYBA MARINE CONSTRUCTION CO/TERRA CONTRACTING ...,NaN,079493355,2,"[ryba, marin, construct, coterra, contract, a,...","ryba,marin,construct,coterra,contract,a,joint,..."
4,6800_6800_0001_0_EPS21602_0,CONT_AWD_0001_6800_EPS21602_6800,68,6800,68HE02,068,6800,68R000,080265258,CAT-ENSOFTEK,NaN,080265258,3,[catensoftek],catensoftek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18914,7008_-NONE-_HSCG3216PSMA006_P00001_-NONE-_0,CONT_AWD_HSCG3216PSMA006_7008_-NONE-_-NONE-,70,7008,00032,070,7008,00032,785365693,"FLO-TECH, L.L.C.",NaN,785365693,27651,[flotech],flotech
18915,9700_-NONE-_SPE4A716D5005_0_-NONE-_-NONE-,CONT_IDV_SPE4A716D5005_9700,97,97AS,SPE4A7,097,97AS,SPE4A7,089107726,"PRECI-MANUFACTURING, INCORPORATED",NaN,089107726,9467,"[precimanufactur, incorpor]","precimanufactur,incorpor"
18916,3600_-NONE-_VA25815P0001_P00002_-NONE-_0,CONT_AWD_VA25815P0001_3600_-NONE-_-NONE-,36,3600,00258,036,3600,00678,968237433,GARY BLUEMKE,NaN,968237433,153461,"[gari, bluemk]","gari,bluemk"
18917,9700_-NONE-_SPE4A716D0005_0_-NONE-_-NONE-,CONT_IDV_SPE4A716D0005_9700,97,97AS,SPE4A7,097,97AS,SPE4A7,121360627,"JANEL'S INDUSTRIES, INC.",NaN,121360627,996,"[janel, industri]","janel,industri"


### Step 5a: Normalizing and using recipient_doing_business_as_name column

Names also appear for some rows in recipient_doing_business_as_name. For rows where this appears, this can also be normalized and used in place of normed values in recipient_name column in cases where it may lead to a larger number of matches.

In [15]:
suppliers_unique_rev = data_norm.recipient_doing_business_as_name.dropna().drop_duplicates().reset_index()

suppliers_unique_rev

,index,recipient_doing_business_as_name
0,109,MIT LINCOLN LABORATORY
1,136,BOEING
2,158,U T C
3,224,SOFTWARE ENGINEERING INSTITUTE
4,238,INTERNATIONAL BUSINESS MACHINES
...,...,...
8893,18571,GOLDEN LIVINGCENTER - BRIARWOOD
8894,18756,AVALON PLACE WHARTON
8895,18758,BRIARWOOD NURSING&REHABILITATION
8896,18849,FAIRFIELD INN -MARRIOTT


In [16]:
suppliers_unique_rev['recipient_doing_business_norm'] = suppliers_unique_rev.recipient_doing_business_as_name.apply(remove_punct).\
apply(remove_whitespace).\
apply(normalize_text)

suppliers_unique_rev['recipient_doing_business_str'] = suppliers_unique_rev.recipient_doing_business_norm.agg(lambda x: ','.join(map(str, x)))

In [17]:
data_norm_rev = data_norm.merge(suppliers_unique_rev, how = 'outer', \
                                on = ['recipient_doing_business_as_name'])

In [18]:
suppliers_rev = data_norm_rev[['recipient_name', 'recipient_name_str', 'recipient_doing_business_str']].drop_duplicates().reset_index()

suppliers_business_as_mult = suppliers_rev.recipient_doing_business_str.dropna()\
[suppliers_rev.recipient_doing_business_str.map(suppliers_rev.\
                                                recipient_doing_business_str.value_counts()) > 1].\
drop_duplicates().\
tolist()


In [19]:
suppliers_rev['recipient_str_rev'] = np.where(((suppliers_rev['recipient_doing_business_str'].\
                                               isin(suppliers_business_as_mult))|\
                                               (suppliers_rev['recipient_doing_business_str'].isin(suppliers_unique))), \
suppliers_rev['recipient_doing_business_str'],\
suppliers_rev['recipient_name_str'])


In [20]:
suppliers_rev['recipient_str_rev'].drop_duplicates().shape[0]

146182

This technique offered a slight additional improvement, with the result that **146,182** unique suppliers are found.

In [21]:
suppliers_rev = suppliers_rev[['recipient_str_rev', 'recipient_name']].reset_index()

data_final = data.merge(suppliers_rev, how = 'outer', on = 'recipient_name').compute()

data_final

,contract_transaction_unique_key,contract_award_unique_key,awarding_agency_code,awarding_sub_agency_code,awarding_office_code,funding_agency_code,funding_sub_agency_code,funding_office_code,recipient_duns,recipient_name,recipient_doing_business_as_name,recipient_parent_duns,index,recipient_str_rev
0,4732_-NONE-_GS00Q16AJC0006_0_-NONE-_0,CONT_AWD_GS00Q16AJC0006_4732_-NONE-_-NONE-,47,4732,47QFCA,097,5700,F2VVJ3,827627154,"IST RESEARCH, LLC",NaN,827627154,6880,"ist,research"
1,9700_-NONE-_FA875014C0231_P00009_-NONE-_4,CONT_AWD_FA875014C0231_9700_-NONE-_-NONE-,97,5700,FA8750,097,97AE,HR0011,827627154,"IST RESEARCH, LLC",NaN,827627154,6880,"ist,research"
2,9700_-NONE-_W56PFY16P0118_0_-NONE-_0,CONT_AWD_W56PFY16P0118_9700_-NONE-_-NONE-,97,2100,W56PFY,097,2100,W90UKT,827627154,"IST RESEARCH, LLC",NaN,827627154,6880,"ist,research"
3,9700_-NONE-_FA875014C0256_P00009_-NONE-_4,CONT_AWD_FA875014C0256_9700_-NONE-_-NONE-,97,5700,FA8750,097,97AE,HR0011,827627154,"IST RESEARCH, LLC",NaN,827627154,6880,"ist,research"
4,9700_-NONE-_N0024414C0009_ARZ998_-NONE-_0,CONT_AWD_N0024414C0009_9700_-NONE-_-NONE-,97,9763,S5111A,097,1700,N00244,827627154,"IST RESEARCH, LLC",NaN,827627154,6880,"ist,research"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055,12C2_-NONE-_AG03R6P1568016905_0_-NONE-_0,CONT_AWD_AG03R6P1568016905_12C2_-NONE-_-NONE-,12,12C2,03R6,012,12C2,0351,007978216,"HAHN SUPPLY, INC.",NaN,007978216,147337,"hahn,suppli"
9056,9700_9700_FA462516FG108_0_FA462515A0004_0,CONT_AWD_FA462516FG108_9700_FA462515A0004_9700,97,5700,FA4625,097,5700,F4E32H,031164965,SWEENEY-PHILLIPS & HOLDREN FUNERAL SERVICE INC,NaN,031164965,148354,"sweeneyphillip,holdren,funer,servic"
9057,9700_4732_FA468616FG018_0_GS03F055CA_0,CONT_AWD_FA468616FG018_9700_GS03F055CA_4732,97,5700,FA4686,097,5700,F1H2EE,869288274,"CHAMPLIN TIRE RECYCLING, INC.",NaN,869288274,148366,"champlin,tire,recycl"
9058,3600_-NONE-_VA118A15P0388_0_-NONE-_0,CONT_AWD_VA118A15P0388_3600_-NONE-_-NONE-,36,3600,0010A,036,3600,0010A,610226243,SERENGETI SYSTEMS INC,NaN,610226243,148683,"serengeti,system"


## Step 6: Evaluating accuracy

I assume that the recipient_parent_duns column provides the true unique suppliers in this dataset and evaluate accuracy of the text normalization approach using the recipient_parent_duns column to provide a few different accuracy measures: 

- **Precision** (number of true positives divided by sum of true positives and false positives)
- **Recall** (number of true positives divided by sum of true positives and false negatives)
- **F1 Score** (2*true positives divided by sum of 2*true positives plus false positives and false negatives)

In [30]:
accuracy = data_final[['recipient_parent_duns', 'recipient_str_rev']].drop_duplicates().reset_index()

### Precision

In [31]:
precision = accuracy.recipient_str_rev[accuracy.recipient_str_rev.map(accuracy.recipient_str_rev.value_counts()) == 1].shape[0]/\
(accuracy.recipient_str_rev[accuracy.recipient_str_rev.map(accuracy.recipient_str_rev.value_counts()) == 1].shape[0] +\
accuracy.recipient_str_rev[accuracy.recipient_str_rev.map(accuracy.recipient_str_rev.value_counts()) > 1].shape[0])

print('The precision of this approach is', str(round(precision, 2)), '.')

The precision of this approach is 0.94 .


### Recall

In [32]:
sensitivity = accuracy.recipient_str_rev[accuracy.recipient_str_rev.map(accuracy.recipient_str_rev.value_counts()) == 1].shape[0]/\
(accuracy.recipient_str_rev[accuracy.recipient_str_rev.map(accuracy.recipient_str_rev.value_counts()) == 1].shape[0] +\
accuracy.recipient_parent_duns[accuracy.recipient_parent_duns.map(accuracy.recipient_parent_duns.value_counts()) > 1].shape[0])

print('The recall of this approach is', str(round(sensitivity, 2)), '.')

The recall of this approach is 0.85 .


### F1 Score

In [33]:
f1 = ((accuracy.recipient_str_rev[accuracy.recipient_str_rev.map(accuracy.recipient_str_rev.value_counts()) == 1].shape[0])*2)/\
(((accuracy.recipient_str_rev[accuracy.recipient_str_rev.map(accuracy.recipient_str_rev.value_counts()) == 1].shape[0])*2) +\
 (accuracy.recipient_str_rev[accuracy.recipient_str_rev.map(accuracy.recipient_str_rev.value_counts()) > 1].shape[0]) +\
 (accuracy.recipient_parent_duns[accuracy.recipient_parent_duns.map(accuracy.recipient_parent_duns.value_counts()) > 1].shape[0]))

print('The F1 Score from this approach is', str(round(f1, 2)), '.')
 

The F1 Score from this approach is 0.89 .


### Step 6a: Alternative strategies

Instead of going the route of text normalization, conversion of sentence embeddings could be implemented to allow more flexibility and fine-tuning in the fuzzy matching procedure. With every word transformed into a numeric value, the resulting list per row could be compared to other rows in terms of a threshold cosine similarity or Levenshtein distance and fuzzy matching could be performed that way. The other benefit to this tactic would the ability for the analyst to control the matching threshold value. I initially thought about pursuing this strategy instead of or in addition to the normalization pipeline I ultimately used, but I did not have enough time to think about how to build a function that would do this pairwise calculation quickly between every possible pair of supplier names in this dataset, perhaps through k-means or other naive clustering analysis (resulting category labels could be appended to original df). It would be interesting to see how the accuracy of this strategy would compare to the strategy I ultimately used (the text normalization strategy had high precision but lower recall, while a strategy such as this may have higher recall but lower precision).

## Step 7: Newly answerable questions

With a new code for true unique supplier, an analyst may return to the full dataset to attempt to answer questions such as the following with increased accuracy (these are just examples of the many questions that this data could be put to use for):

- What kinds of suppliers have agreements with the greatest number of offices and what kinds of offices have agreements with the greatest number of suppliers?
- Do these cluster in any sectors (using the sector indicator columns present in the original dataset)?
- Do offices or divisions have multiple agreements with the same supplier under different supplier names? (With multiple year datasets) are offices or divisions recording suppliers as new recipient_names when there is actually a preexisting relationship with the same supplier under a preexisting name?
